# Mendeleev's Periodic Table of Elements

The notebook is inspired by [this](https://plotnine.org/reference/geom_tile#periodic-table-of-elements) and [this](https://docs.bokeh.org/en/latest/docs/examples/topics/categorical/periodic.html) examples.

The data is available under the Creative Commons Attribution-ShareAlike 4.0 International Public License (CC BY-SA 4.0). For more details, see [here](https://github.com/dataexplorer/datasets/blob/master/License.md) or visit [Data Explorer](https://www.data-explorer.com/data/).

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
import org.jetbrains.letsPlot.intern.Feature

In [3]:
fun DataFrame<*>.replaceTypeByElement(element: String, value: String): DataFrame<*> {
    return this.update { "Type"<String?>() }
                   .where { "Element"<String>() == element }
                   .with { value }
}

fun DataFrame<*>.replaceDataByAtomicNumber(atomicNumber: Int, element: String, symbol: String, type: String): DataFrame<*> {
    return this.update { "Element"<String>() }
                   .where { "Atomic Number"<Int>() == atomicNumber }
                   .with { element }
               .update { "Symbol"<String>() }
                   .where { "Atomic Number"<Int>() == atomicNumber }
                   .with { symbol }
               .update { "Type"<String?>() }
                   .where { "Atomic Number"<Int>() == atomicNumber }
                   .with { type }
}

fun getElementsData(): DataFrame<*> {
    val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/chemical_elements.csv")
        .replaceTypeByElement("Francium", "Alkali Metal")
        .replaceTypeByElement("Radium", "Alkaline Earth Metal")
        .replaceTypeByElement("Astatine", "Halogen")
        .replaceTypeByElement("Radon", "Noble Gas")
        .replaceDataByAtomicNumber(113, element = "Nihonium", symbol = "Nh", type = "Metal")
        .replaceDataByAtomicNumber(114, element = "Flerovium", symbol = "Fl", type = "Metal")
        .replaceDataByAtomicNumber(115, element = "Moscovium", symbol = "Mc", type = "Metal")
        .replaceDataByAtomicNumber(116, element = "Livermorium", symbol = "Lv", type = "Metal")
        .replaceDataByAtomicNumber(117, element = "Tennessine", symbol = "Ts", type = "Halogen")
        .replaceDataByAtomicNumber(118, element = "Oganesson", symbol = "Og", type = "Noble Gas")
        .update { "Type"<String?>() }
            .where { "Type"<String?>() == "Transactinide" }
            .with { "Transition Metal" }
    return df
}

fun prepareTopDf(df: DataFrame<*>): DataFrame<*> {
    return df.filter { "Type"<String?>() !in setOf("Actinide", "Lanthanide") }
             .add("X") { "Group"<Int>() }
             .add("Y") { "Period"<Int>() }
}

fun prepareBottomDf(df: DataFrame<*>): DataFrame<*> {
    val filteredDf = df.filter { "Type"<String?>() in setOf("Actinide", "Lanthanide") }
    val nRows = 2
    val nCols = filteredDf.rowsCount() / nRows
    val hShift = 3
    val vShift = 2.5
    return filteredDf
        .add("X") { index() % nCols + hShift }
        .add("Y") { "Period"<Int>() + vShift }
}

fun getExtraTopDf(): DataFrame<*> {
    return dataFrameOf(
        "X" to listOf(3, 3),
        "Y" to listOf(6, 7),
        "Type" to listOf("Lanthanide", "Actinide"),
        "Range" to listOf("57-71", "89-103"),
    )
}

fun getTableKeyDf(df: DataFrame<*>, x: Double, y: Double, atomicNumber: Int): DataFrame<*> {
    return df.filter { "Atomic Number"<Int>() == atomicNumber }
             .add("X") { x }
             .add("Y") { y }
}

fun getGroupDf(df: DataFrame<*>): DataFrame<*> {
    return df.groupBy("Group")
             .aggregate {
                 min { "Period"<Int>() } into "Y"
             }
}

fun getPeriodDf(minValue: Int, maxValue: Int): DataFrame<*> {
    return dataFrameOf(
        "X" to List(maxValue - minValue + 1) { 0 },
        "Period" to (minValue..maxValue).toList()
    )
}

fun getAnnotationsDf(x: Double, y: Double): DataFrame<*> {
    return dataFrameOf(
        "X" to listOf(x + 0.8, x + 1.0, x + 0.4),
        "Y" to listOf(y - 0.9, y + 0.1, y + 1.0),
        "Label" to listOf("Atomic Number", "Symbol", "Atomic Mass")
    )
}

In [4]:
val elementsDf = getElementsData()
elementsDf.head()

Atomic Number,Element,Symbol,Atomic Weight,Period,Group,Phase,Most Stable Crystal,Type,Ionic Radius,Atomic Radius,Electronegativity,First Ionization Potential,Density,Melting Point (K),Boiling Point (K),Isotopes,Discoverer,Year of Discovery,Specific Heat Capacity,Electron Configuration,Display Row,Display Column
1,Hydrogen,H,1.007940,1,1,gas,null,Nonmetal,0.012000,0.790000,2.200000,13.598400,0.000090,14.175000,20.280000,3,Cavendish,1766,14.304000,1s1,1,1
2,Helium,He,4.002602,1,18,gas,null,Noble Gas,null,0.490000,null,24.587400,0.000179,null,4.220000,5,Janssen,1868,5.193000,1s2,1,18
3,Lithium,Li,6.941000,2,1,solid,bcc,Alkali Metal,0.760000,2.100000,0.980000,5.391700,0.534000,453.850000,1615.000000,5,Arfvedson,1817,3.582000,[He] 2s1,2,1
4,Beryllium,Be,9.012182,2,2,solid,hex,Alkaline Earth Metal,0.350000,1.400000,1.570000,9.322700,1.850000,1560.150000,2742.000000,6,Vaulquelin,1798,1.825000,[He] 2s2,2,2
5,Boron,B,10.811000,2,13,solid,rho,Metalloid,0.230000,1.200000,2.040000,8.298000,2.340000,2573.150000,4200.000000,6,Gay-Lussac,1808,1.026000,[He] 2s2 2p1,2,13


In [5]:
val tileSide = 0.95
val tileRatio = 1.2
val tableKeySizeRatio = 1.5
val tableKeyX = 10.25
val tableKeyY = 1.25

val topDf = prepareTopDf(elementsDf)
val bottomDf = prepareBottomDf(elementsDf)
val extraTopDf = getExtraTopDf()
val tableKeyDf = getTableKeyDf(elementsDf, tableKeyX, tableKeyY, atomicNumber = 78)
val groupDf = getGroupDf(topDf)
val periodDf = getPeriodDf(1, 7)
val annotationsDf = getAnnotationsDf(tableKeyX, tableKeyY)

In [6]:
fun innerText(df: DataFrame<*>, ratio: Double = 1.0): Feature {
    return if ("Range" in df.columnNames()) {
        geomText(data = df.toMap(), nudgeY = 0.05 * ratio, size = 5 * ratio, fontface = "bold")
            { label = "Range" } +
        geomText(data = df.toMap(), nudgeY = -0.2 * ratio, size = 4 * ratio)
            { label = "Type" }
    } else {
        geomText(data = df.toMap(), nudgeX = -0.37 * ratio, nudgeY = 0.37 * ratio, hjust = "left", vjust = "top", size = 5 * ratio)
            { label = "Atomic Number" } +
        geomText(data = df.toMap(), nudgeY = 0.05 * ratio, size = 7 * ratio, fontface = "bold")
            { label = "Symbol" } +
        geomText(data = df.toMap(), nudgeY = -0.2 * ratio, size = 4 * ratio, labelFormat = ".3~f")
            { label = "Atomic Weight" }
    }
}

fun tableKeyAnnotations(x: Double, y: Double): Feature {
    val tableKeyArrow = arrow(angle = 30, length = 4, type = "closed")
    return geomCurve(x = x + 0.7, y = y - 0.9, xend = x - 0.3, yend = y - 0.6, curvature = 0.4, ncp = 1, arrow = tableKeyArrow) +
        geomSegment(x = x + 0.9, y = y + 0.1, xend = x + 0.3, yend = y - 0.1, arrow = tableKeyArrow) +
        geomCurve(x = x + 0.3, y = y + 1.0, xend = x, yend = y + 0.5, curvature = -0.4, ncp = 1, arrow = tableKeyArrow)
}

val elementTooltips = layerTooltips().title("@Element\n(@Type)")
                                     .line("@|@{Atomic Number}")
                                     .line("Atomic Mass|@{Atomic Weight}")
                                     .line("@|@{Electron Configuration}")
val tableTheme = theme(plotTitle = elementText(size = 26, face = "bold", margin = listOf(30, 0, 5, 0), hjust = 0.5),
                       plotCaption = elementText(size = 18),
                       plotBackground = elementRect(color = "black", size = 3),
                       legendBackground = "blank").legendPosition(0.36, 0.85)

letsPlot { x = "X"; y = "Y"; fill = "Type" } +
    geomTile(data = topDf.toMap(), color = "black", size = 0.25,
             width = tileSide, height = tileSide, tooltips = elementTooltips) +
    geomTile(data = extraTopDf.toMap(), color = "black", size = 0.25,
             width = tileSide, height = tileSide, tooltips = tooltipsNone) +
    geomTile(data = bottomDf.toMap(), color = "black", size = 0.25,
             width = tileSide, height = tileSide, tooltips = elementTooltips) +
    geomTile(data = tableKeyDf.toMap(), color = "black", size = 0.25,
             width = tableKeySizeRatio * tileSide, height = tableKeySizeRatio * tileSide, tooltips = tooltipsNone) +
    innerText(topDf) +
    innerText(extraTopDf) +
    innerText(bottomDf) +
    innerText(tableKeyDf, ratio = tableKeySizeRatio) +
    geomText(data = groupDf.toMap(), color = "gray", nudgeY = 0.525, vjust = "bottom", size = 6)
        { x = "Group"; y = "Y"; label = "Group" } +
    geomText(data = periodDf.toMap(), color = "gray", nudgeX = 0.375, vjust = "right", size = 6)
        { x = "X"; y = "Period"; label = "Period" } +
    geomText(data = annotationsDf.toMap(), hjust = 0) { label = "Label" } +
        tableKeyAnnotations(tableKeyX, tableKeyY) +
    scaleYReverse() +
    scaleFillBrewer(name = "", type = "qual", palette = "Set2", guide = guideLegend(ncol = 2)) +
    coordFixed(ratio = tileRatio) +
    labs(title = "Periodic Table of Chemical Elements", caption = "© 1869, Dmitri Mendeleev") +
    ggsize(1000, 700) +
    themeVoid() + tableTheme

1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 11 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 13 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 17 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 19 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 21 
 
 
 
 
 
 
 22 
 
 
 
 
 
 
 23 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 26 
 
 
 
 
 
 
 27 
 
 
 
 
 
 
 28 
 
 
 
 
 
 
 29 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 31 
 
 
 
 
 
 
 32 
 
 
 
 
 
 
 33 
 
 
 
 
 
 
 34 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 36 
 
 
 
 
 
 
 37 
 
 
 
 
 
 
 38 
 
 
 
 
 
 
 39 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 41 
 
 
 
 
 
 
 42 
 
 
 
 
 
 
 43 
 
 
 
 
 
 
 44 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 46 
 
 
 
 
 
 
 47 
 
 
 
 
 
 
 48 
 
 
 
 
 
 
 49 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 51 
 
 
 
 
 
 
 52 
 
 
 
 
 
 
 53 
 
 
 
 
 
 
 54 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 56 
 
 
 
 
 
 
 72 
 
 
 
 
 
 
 73 
 
 
 
 
 
 
 74 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 76 
 
 
 
 
 
 
 77 
 
 
 
 
 
 
 78 
 
 
 
 
 
 
 79 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 81 
 
 
 
 
 
 
 82 
 
 
 
 
 
 
 83 
 
 
 
 
 
 
 84 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 86 
 
 
 
 
 
 
 87 
 
 
 
 
 
 
 88 
 
 
 
 
 
 
 104 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 106 
 
 
 
 
 
 
 107 
 
 
 
 
 
 
 108 
 
 
 
 
 
 
 109 
 
 
 
 
 
 
 110 
 
 
 
 
 
 
 111 
 
 
 
 
 
 
 112 
 
 
 
 
 
 
 113 
 
 
 
 
 
 
 114 
 
 
 
 
 
 
 115 
 
 
 
 
 
 
 116 
 
 
 
 
 
 
 117 
 
 
 
 
 
 
 118 
 
 
 
 
 
 
 
 
 H 
 
 
 
 
 
 
 He 
 
 
 
 
 
 
 Li 
 
 
 
 
 
 
 Be 
 
 
 
 
 
 
 B 
 
 
 
 
 
 
 C 
 
 
 
 
 
 
 N 
 
 
 
 
 
 
 O 
 
 
 
 
 
 
 F 
 
 
 
 
 
 
 Ne 
 
 
 
 
 
 
 Na 
 
 
 
 
 
 
 Mg 
 
 
 
 
 
 
 Al 
 
 
 
 
 
 
 Si 
 
 
 
 
 
 
 P 
 
 
 
 
 
 
 S 
 
 
 
 
 
 
 Cl 
 
 
 
 
 
 
 Ar 
 
 
 
 
 
 
 K 
 
 
 
 
 
 
 Ca 
 
 
 
 
 
 
 Sc 
 
 
 
 
 
 
 Ti 
 
 
 
 
 
 
 V 
 
 
 
 
 
 
 Cr 
 
 
 
 
 
 
 Mn 
 
 
 
 
 
 
 Fe 
 
 
 
 
 
 
 Co 
 
 
 
 
 
 
 Ni 
 
 
 
 
 
 
 Cu 
 
 
 
 
 
 
 Zn 
 
 
 
 
 
 
 Ga 
 
 
 
 
 
 
 Ge 
 
 
 
 
 
 
 As 
 
 
 
 
 
 
 Se 
 
 
 
 
 
 
 Br 
 
 
 
 
 
 
 Kr 
 
 
 
 
 
 
 Rb 
 
 
 
 
 
 
 Sr 
 
 
 
 
 
 
 Y 
 
 
 
 
 
 
 Zr 
 
 
 
 
 
 
 Nb 
 
 
 
 
 
 
 Mo 
 
 
 
 
 
 
 Tc 
 
 
 
 
 
 
 Ru 
 
 
 
 
 
 
 Rh 
 
 
 
 
 
 
 Pd 
 
 
 
 
 
 
 Ag 
 
 
 
 
 
 
 Cd 
 
 
 
 
 
 
 In 
 
 
 
 
 
 
 Sn 
 
 
 
 
 
 
 Sb 
 
 
 
 
 
 
 Te 
 
 
 
 
 
 
 I 
 
 
 
 
 
 
 Xe 
 
 
 
 
 
 
 Cs 
 
 
 
 
 
 
 Ba 
 
 
 
 
 
 
 Hf 
 
 
 
 
 
 
 Ta 
 
 
 
 
 
 
 W 
 
 
 
 
 
 
 Re 
 
 
 
 
 
 
 Os 
 
 
 
 
 
 
 Ir 
 
 
 
 
 
 
 Pt 
 
 
 
 
 
 
 Au 
 
 
 
 
 
 
 Hg 
 
 
 
 
 
 
 Tl 
 
 
 
 
 
 
 Pb 
 
 
 
 
 
 
 Bi 
 
 
 
 
 
 
 Po 
 
 
 
 
 
 
 At 
 
 
 
 
 
 
 Rn 
 
 
 
 
 
 
 Fr 
 
 
 
 
 
 
 Ra 
 
 
 
 
 
 
 Rf 
 
 
 
 
 
 
 Db 
 
 
 
 
 
 
 Sg 
 
 
 
 
 
 
 Bh 
 
 
 
 
 
 
 Hs 
 
 
 
 
 
 
 Mt 
 
 
 
 
 
 
 Ds 
 
 
 
 
 
 
 Rg 
 
 
 
 
 
 
 Cn 
 
 
 
 
 
 
 Nh 
 
 
 
 
 
 
 Fl 
 
 
 
 
 
 
 Mc 
 
 
 
 
 
 
 Lv 
 
 
 
 
 
 
 Ts 
 
 
 
 
 
 
 Og 
 
 
 
 
 
 
 
 
 1.008 
 
 
 
 
 
 
 4.003 
 
 
 
 
 
 
 6.941 
 
 
 
 
 
 
 9.012 
 
 
 
 
 
 
 10.811 
 
 
 
 
 
 
 12.011 
 
 
 
 
 
 
 14.007 
 
 
 
 
 
 
 15.999 
 
 
 
 
 
 
 18.998 
 
 
 
 
 
 
 20.18 
 
 
 
 
 
 
 22.99 
 
 
 
 
 
 
 24.305 
 
 
 
 
 
 
 26.982 
 
 
 
 
 
 
 28.086 
 
 
 
 
 
 
 30.974 
 
 
 
 
 
 
 32.065 
 
 
 
 
 
 
 35.453 
 
 
 
 
 
 
 39.948 
 
 
 
 
 
 
 39.098 
 
 
 
 
 
 
 40.078 
 
 
 
 
 
 
 44.956 
 
 
 
 
 
 
 47.867 
 
 
 
 
 
 
 50.942 
 
 
 
 
 
 
 51.996 
 
 
 
 
 
 
 54.938 
 
 
 
 
 
 
 55.845 
 
 
 
 
 
 
 58.933 
 
 
 
 
 
 
 58.693 
 
 
 
 
 
 
 63.546 
 
 
 
 
 
 
 65.38 
 
 
 
 
 
 
 69.723 
 
 
 
 
 
 
 72.64 
 
 
 
 
 
 
 74.922 
 
 
 
 
 
 
 78.96 
 
 
 
 
 
 
 79.904 
 
 
 
 
 
 
 83.798 
 
 
 
 
 
 
 85.468 
 
 
 
 
 
 
 87.62 
 
 
 
 
 
 
 88.906 
 
 
 
 
 
 
 91.224 
 
 
 
 
 
 
 92.906 
 
 
 
 
 
 
 95.96 
 
 
 
 
 
 
 98 
 
 
 
 
 
 
 101.07 
 
 
 
 
 
 
 102.906 
 
 
 
 
 
 
 106.42 
 
 
 
 
 
 
 107.868 
 
 
 
 
 
 
 112.411 
 
 
 
 
 
 
 114.818 
 
